### Connect to Milvus Server

In [1]:
from pymilvus import connections
connections.connect(
  alias="default",
  uri="http://localhost:19530",
)

### Create a Collection

In [3]:
from pymilvus import CollectionSchema, FieldSchema, DataType
book_id = FieldSchema(
  name="book_id",
  dtype=DataType.INT64,
  is_primary=True,
)
book_name = FieldSchema(
  name="book_name",
  dtype=DataType.VARCHAR,
  max_length=200,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value="Unknown"
)
word_count = FieldSchema(
  name="word_count",
  dtype=DataType.INT64,
  # The default value will be used if this field is left empty during data inserts or upserts.
  # The data type of `default_value` must be the same as that specified in `dtype`.
  default_value=9999
)
book_intro = FieldSchema(
  name="book_intro",
  dtype=DataType.FLOAT_VECTOR,
  dim=2
)
schema = CollectionSchema(
  fields=[book_id, book_name, word_count, book_intro],
  description="Test book search",
  enable_dynamic_field=True
)
collection_name = "book"



In [4]:
from pymilvus import Collection
collection = Collection(
    name=collection_name,
    schema=schema,
    using='default',
    shards_num=2
    )

### Insert Data into Milvus



In [5]:
import random
data = [
  [i for i in range(2000)],
  [str(i) for i in range(2000)],
  [i for i in range(10000, 12000)],
  [[random.random() for _ in range(2)] for _ in range(2000)],
  # use `default_value` for a field
  [], 
  # or
  None,
  # or just omit the field
]

data.append([str("dy"*i) for i in range(2000)])


In [6]:
from pymilvus import Collection
collection = Collection(collection_name)      # Get an existing collection.
mr = collection.insert(data[:4])  # Insert only the first 4 lists that match the schema fields.


### Build an index on vectors


In [7]:
index_params = {
  "metric_type":"L2",
  "index_type":"IVF_FLAT",
  "params":{"nlist":1024}
}


In [8]:
from pymilvus import Collection, utility
collection = Collection("book")      
collection.create_index(
  field_name="book_intro", 
  index_params=index_params
)

utility.index_building_progress("book")


{'total_rows': 0, 'indexed_rows': 0, 'pending_index_rows': 0}

### Vector Similarity search


In [9]:
collection.load()

In [10]:
search_params = {
    "metric_type": "L2", 
    "offset": 0, 
    "ignore_growing": False, 
    "params": {"nprobe": 10}
}


In [11]:
results = collection.search(
    data=[[0.1, 0.2]], 
    anns_field="book_intro", 
    # the sum of `offset` in `param` and `limit` 
    # should be less than 16384.
    param=search_params,
    limit=10,
    expr=None,
    # set the names of the fields you want to 
    # retrieve from the search result.
    output_fields=['title'],
    consistency_level="Strong"
)

results[0].ids

results[0].distances

hit = results[0][0]
hit.entity.get('title')


In [12]:
results[0].ids
results[0].distances


[0.0007631918415427208,
 0.0008136157994158566,
 0.0008455310598947108,
 0.0008900082902982831,
 0.0009001400321722031,
 0.0010042081121355295,
 0.001197516219690442,
 0.0012166542001068592,
 0.0015185587108135223,
 0.0018159594619646668]

### Query Vector

In [13]:
res = collection.query(
  expr = "book_id in [2,4,6,8]",
  offset = 0,
  limit = 10, 
  output_fields = ["book_id", "book_intro"],
)


In [14]:
sorted_res = sorted(res, key=lambda k: k['book_id'])
sorted_res


[{'book_id': 2, 'book_intro': [0.31622496, 0.91709125]},
 {'book_id': 4, 'book_intro': [0.91241616, 0.8291225]},
 {'book_id': 6, 'book_intro': [0.34510216, 0.16219896]},
 {'book_id': 8, 'book_intro': [0.008409258, 0.56888074]}]

In [15]:
# Disconnect from Milvus server
connections.disconnect("default")